In [51]:
import copy
import os
import random

import importlib_resources
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm

from cmmrt.projection.data import get_representatives
from cmmrt.projection.data import *
from cmmrt.rt.predictions import load_cmm_predictions
from cmmrt.projection.models.projector.loader import _load_projector_pipeline_from

In [52]:
def get_ppm_error(mass, ppm_error=10):
    return (round(mass) * ppm_error) / 10 ** 6

In [53]:
def rank_data(train_df, nreps, nid_metabolites):
    index_list = range(0, train_df.shape[0])
    for i in range(0, nreps):
        ii = random.sample(index_list, nid_metabolites)
        train = train_df.iloc[ii]
        id_remain = np.setdiff1d(index_list, ii)
        to_rank = train_df.iloc[id_remain]
        print(' '.join(train['Name']))
        x, y = (
            torch.from_numpy(train.prediction.values.reshape(-1, 1)),
            torch.from_numpy(train.rt.values*60)
                )
        projector.projector.prepare_metatesting()
        projector.fit(x, y)
        mass_error_seed = 123
        if mass_error_seed is not None:
            np.random.seed(mass_error_seed)
            
        candidates_list = []

        for index, row in to_rank.iterrows():
            # Skip if the compound is not in the test set (since it wouldn't have a chance to be in the top results)
            error = get_ppm_error(row.calc_mw)
            candidates = predicted_pubchem[
                (predicted_pubchem["ExactMass"] >= (row.calc_mw - error))
                & (predicted_pubchem["ExactMass"] <= (row.calc_mw + error))
                ].copy()

            candidates = candidates.drop(['Unnamed: 0', 'MolecularWeight', 'cmm_id'], axis=1)
            candidates = candidates.rename(columns={'prediction':'rt_predicted'})

            if candidates.shape[0] > 0:
                candidates['FeatureID'] = row.FeatureID
                candidates['rt_experimental'] = row.rt*60
                candidates['mass_experimental'] = row.calc_mw
                candidates['z_score'] = pd.NA
                candidates['mass_error'] = abs(candidates.ExactMass - row.calc_mw)
                # add small noise to unbreak ties
                candidates['mass_error'] = candidates['mass_error'] + np.random.uniform(0, 1e-6, candidates.shape[0])
                candidates.sort_values(by='mass_error', inplace=True)
                scores = projector.z_score(candidates[['rt_predicted']].values, np.array([row.rt*60]))
                scores = scores.cpu().numpy()
                candidates.loc[:, 'z_score'] = scores
                candidates['z_score'] = pd.to_numeric(candidates['z_score'], errors = 'coerce')
                candidates.sort_values("z_score", inplace=True)
                candidates = candidates.nlargest(3, ['z_score'])
                candidates_list.append(candidates)

        candidates_final = pd.concat(candidates_list).reset_index(drop=True)
        candidates_final = candidates_final[['FeatureID', 'mass_experimental', 'rt_experimental', 
                                             'rt_predicted', 'mass_error', 'z_score', 'Title', 'MolecularFormula',
                                             'ExactMass', 'InChIKey', 'InChI', 'pid']]

        candidates_final.to_csv(f'results/results_test_loop/Candidate_annotation_nannot_{str(nid_metabolites)}_rep_{str(i)}.csv', 
                                index=False)
        
        # plotting
        sorted_x = torch.arange(x.min() - 0.5, x.max() + 0.5, 0.1, dtype=torch.float32)
        plt.scatter(predicted_pubchem.prediction.values,
                    projector.predict(predicted_pubchem.prediction.values)[0])
        preds_mean, lb, ub = projector.predict(sorted_x)
        plt.scatter(x, y, marker='x')
        plt.fill_between(sorted_x, lb, ub, alpha=0.2, color='orange')
        plt.plot(sorted_x, preds_mean, color='orange')
        plt.title(f'N_annotated {str(nid_metabolites)}, rep {str(i)}')
        plt.xlabel("Predicted RT")
        plt.ylabel("Projected/Experimental RT")
        with torch.no_grad():
            sorted_x_ = torch.from_numpy(projector.x_scaler.transform(sorted_x.numpy().reshape(-1, 1)))
            tmp = projector.projector.gp.mean_module(sorted_x_)
            tmp = projector.y_scaler.inverse_transform(tmp.reshape(-1, 1)).flatten()
            plt.plot(sorted_x, tmp, color='green')
        plt.savefig(f'results/results_test_loop/Candidate_plot_nannot_{str(nid_metabolites)}_rep_{str(i)}.png')
        plt.close()

## Adding own data

In [54]:
pubchem_db = pd.read_csv('data/final_pubchem_results.csv')
pubchem_db = pubchem_db.astype({'CID': 'str'})

predicted_pubchem = pd.read_csv('results/predicted_rt_db.csv', )
predicted_pubchem = predicted_pubchem.astype({'pid': 'str'})
predicted_pubchem = predicted_pubchem.merge(pubchem_db, left_on='pid', right_on='CID', how='left')

,Unnamed: 0,pid,cmm_id,prediction,CID,Title,MolecularFormula,MolecularWeight,ExactMass,InChIKey,InChI
80035,80033,97733655,80033,942.59216,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80036,80034,98666786,80034,663.49090,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80037,80035,98670835,80035,649.48760,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80038,80036,98779314,80036,802.17440,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80039,80037,99905419,80037,603.21560,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#Filtering out the CID

In [55]:
data_to_rank = pd.read_csv('data/RP_skin_all.csv')
data_annotated = pd.read_csv('data/RP_skin_id.csv')
data_annotated = data_annotated.astype({'CID': 'str'})
#data_annotated = data_annotated.merge(pubchem_db, left_on='Name', right_on='Title', how='left')
data_to_rank.head()


,FeatureID,Name,Formula,calc_mw,mz,rt,annot_source,Annot. Source: mzVault Search,CID
0,Feature2369,Indole-3-carboxaldehyde,C9 H7 N O,145.05268,146.05996,12.402,mzVault and mzCloud,Full match,10256
1,Feature2367,Ornithine,C5 H12 N2 O2,132.08982,133.09709,1.058,mzVault and mzCloud,Full match,6262
2,Feature2365,D-Aspartic acid,C4 H7 N O4,133.03737,134.04465,1.215,mzVault and mzCloud,Full match,83887
3,Feature2364,Cytidine-5'-monophosphate,C9 H14 N3 O8 P,323.05170,324.05897,1.799,mzVault and mzCloud,Full match,6131
4,Feature2363,D-Histidine,C6 H9 N3 O2,155.06937,156.07667,1.131,mzVault and mzCloud,Full match,71083


In [56]:
filtered_pids.head()

,CID
0,5139
1,3505
2,2159
3,1340
4,3344


In [57]:
#still testing
SMMRT_Descriptors_filtered = pd.read_csv('filtered_data/SMRT_descriptors_filtered.csv')
SMMRT_Descriptors_filtered.rename(columns={'...1':'CID'}, inplace= True)
SMMRT_Descriptors_filtered.head()

,CID,pid,rt,MW,AMW,Sv,Se,Sp,Si,Mv,...,s1_numAroBonds,s2_numAroBonds,s3_numAroBonds,s4_numAroBonds,s34_size,s34_relSize,s34_phSize,s34_phRelSize,chiralMoment,chiralPhMoment
0,0,5139,93.5,104.20,7.442857,7.8103,13.9308,8.9433,16.1622,0.557879,...,0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000
1,1,3505,687.8,414.37,7.968654,32.1852,52.5379,34.2332,58.9935,0.618946,...,0,0.0,0.0,6.0,20.0,0.740741,5.0,0.185185,19.131126,4.472136
2,2,2159,590.7,369.54,7.106538,30.4321,52.2942,32.6196,59.2349,0.585233,...,0,0.0,0.0,6.0,22.0,0.880000,7.0,0.280000,28.284271,9.949874
3,3,1340,583.6,161.17,8.482632,13.0314,19.4072,13.1989,21.1627,0.685863,...,0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000
4,4,3344,579.0,260.37,6.676154,23.2136,38.8106,24.7730,44.1522,0.595221,...,0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000


In [68]:
cid = filtered_pids[filtered_pids['CID'] == 23327]
cid.head()

,CID
211,23327


: 

In [65]:
#ANother testing block
#filtered_df = second_df[second_df['pid'].isin(first_df['pid'])]

#filtered_pids['CID'] = filtered_pids['CID'].astype('object')
data_annotated['CID'] = data_annotated['CID'].astype('int64')

filtered_df = data_annotated[data_annotated['CID'].isin(filtered_pids['CID'])]


filtered_df.head()


,FeatureID,Name,Formula,calc_mw,mz,rt,annot_source,Annot. Source: mzVault Search,CID
16,Feature2347,D-Glutamic acid,C5 H9 N O4,147.05302,148.06029,1.286,mzVault and mzCloud,Full match,23327
32,Feature2321,D-Glutamic acid,C5 H9 N O4,147.05304,148.06032,1.492,mzVault and mzCloud,Full match,23327


In [59]:
projector = _load_projector_pipeline_from(f"../cmmrt/cmmrt/data/metalearned_projectors/p2e_rl.pt", mean='constant',
                                          kernel='rbf+linear')
pubchems = np.array(data_annotated.CID)

/home/ernesto12/.local/lib/python3.10/site-packages/gpytorch-1.10-py3.10.egg/gpytorch/means/constant_mean.py:20: OldVersionWarning: You have loaded a GP model with a ConstantMean  from a previous version of GPyTorch. The mean module parameter `constant` has been renamed to `raw_constant`. Additionally, the shape of `raw_constant` is now *batch_shape, whereas the shape of `constant` was *batch_shape x 1. We have updated the name/shape of the parameter in your state dict, but we recommend that you re-save your model.
  warnings.warn(


In [60]:
train_logical = data_annotated['CID'].isin(pubchems)
train = data_annotated[train_logical]
train = train.merge(predicted_pubchem, on='CID', how='left').dropna()

In [61]:
rank_data(train_df = train, nreps=10, nid_metabolites=10)

ValueError: Sample larger than population or is negative

In [ ]:
rank_data(train_df = train, nreps=10, nid_metabolites=20)

NameError: name 'rank_data' is not defined

In [ ]:
rank_data(train_df = train, nreps=10, nid_metabolites=30)

L-Carnitine trans-Urocanic Acid Allantoin Citicoline Uridine-5'-phosphate Thymine Cytidine L-Carnitine sn-glycero-3-Phosphocholine Creatinine L-Carnitine trans-Urocanic Acid D-Glutamine Glycyl-L-leucine N-Acetylglutamic acid L-Carnitine Adenosine L(-)-Pipecolinic acid L-Carnitine trans-Urocanic Acid D-Valine L-Carnitine D-Glutamic acid L-Carnitine Uric acid Indole-3-carboxaldehyde L-Citrulline L-Carnitine 2-Deoxy-D-glucose Xanthine


/home/ernesto12/.local/lib/python3.10/site-packages/linear_operator-0.4.0-py3.10.egg/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  warnings.warn(
/home/ernesto12/.local/lib/python3.10/site-packages/linear_operator-0.4.0-py3.10.egg/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-05 to the diagonal
  warnings.warn(
/home/ernesto12/.local/lib/python3.10/site-packages/linear_operator-0.4.0-py3.10.egg/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  warnings.warn(
/home/ernesto12/.local/lib/python3.10/site-packages/linear_operator-0.4.0-py3.10.egg/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-05 to the diagonal
  warnings.warn(
/home/ernesto12/.local/lib/python3.10/site-packages/linear_operator-0.4.0-py3.10.egg/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added j

D-Serine sn-glycero-3-Phosphocholine Maltotriose L-Cystine L-Carnitine L-Carnitine trans-Urocanic Acid trans-Urocanic Acid L-Carnitine Allantoin D-Glutamic acid D-Proline Cytidine-5'-monophosphate N-Acetylneuraminic acid Uridine Trigonelline Nicotinic acid Choline Creatinine D-Glutamine Phosphocholine L-Carnitine L-Carnitine L-Tyrosine L(-)-Pipecolinic acid L-Norleucine N-Acetylglutamic acid Ornithine L-Carnitine Indole-3-carboxaldehyde


/home/ernesto12/.local/lib/python3.10/site-packages/linear_operator-0.4.0-py3.10.egg/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  warnings.warn(
/home/ernesto12/.local/lib/python3.10/site-packages/linear_operator-0.4.0-py3.10.egg/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-05 to the diagonal
  warnings.warn(
/home/ernesto12/.local/lib/python3.10/site-packages/linear_operator-0.4.0-py3.10.egg/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  warnings.warn(
/home/ernesto12/.local/lib/python3.10/site-packages/linear_operator-0.4.0-py3.10.egg/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-05 to the diagonal
  warnings.warn(
/home/ernesto12/.local/lib/python3.10/site-packages/linear_operator-0.4.0-py3.10.egg/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added j

2-Pyrrolidinone Creatine Uridine L-Carnitine trans-Urocanic Acid Betaine Choline Creatine Creatine L-Alanine D-Threonine Trigonelline L-Carnitine L-Carnitine Ornithine Hippuric acid D-Glutamic acid D-Proline L-Carnitine L-Carnitine D-Glutamic acid L-Carnitine Phenylac-Gly-OH D-Leucine N-Acetylneuraminic acid L-Citrulline L-Carnitine L-Carnitine L-Norleucine N-Acetylneuraminic acid


...

/home/ernesto12/.local/lib/python3.10/site-packages/linear_operator-0.4.0-py3.10.egg/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  warnings.warn(
/home/ernesto12/.local/lib/python3.10/site-packages/linear_operator-0.4.0-py3.10.egg/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-05 to the diagonal
  warnings.warn(
/home/ernesto12/.local/lib/python3.10/site-packages/linear_operator-0.4.0-py3.10.egg/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  warnings.warn(
/home/ernesto12/.local/lib/python3.10/site-packages/linear_operator-0.4.0-py3.10.egg/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-05 to the diagonal
  warnings.warn(
/home/ernesto12/.local/lib/python3.10/site-packages/linear_operator-0.4.0-py3.10.egg/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added j

N-Acetyl-D-mannosamine Maltotriose L-Carnitine L-Carnitine Cytidine-5'-monophosphate L-Alanine L-Carnitine Nicotinic acid Trigonelline L-Carnitine Phosphocholine L-Norleucine trans-Urocanic Acid L-Carnosine Uridine-5'-phosphate L-Alanine L-Citrulline Indole-3-carboxaldehyde Choline D-Glutamine L-Norleucine Uric acid trans-Urocanic Acid L-Carnitine Phenylac-Gly-OH N-Acetylglycine trans-Urocanic Acid 2-Deoxy-D-glucose trans-Urocanic Acid sn-glycero-3-Phosphocholine


/home/ernesto12/.local/lib/python3.10/site-packages/linear_operator-0.4.0-py3.10.egg/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  warnings.warn(
/home/ernesto12/.local/lib/python3.10/site-packages/linear_operator-0.4.0-py3.10.egg/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-05 to the diagonal
  warnings.warn(
/home/ernesto12/.local/lib/python3.10/site-packages/linear_operator-0.4.0-py3.10.egg/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  warnings.warn(
/home/ernesto12/.local/lib/python3.10/site-packages/linear_operator-0.4.0-py3.10.egg/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-05 to the diagonal
  warnings.warn(
/home/ernesto12/.local/lib/python3.10/site-packages/linear_operator-0.4.0-py3.10.egg/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added j

L-Citrulline trans-Urocanic Acid L-Norleucine Bicine D-Valine L-Carnitine Allantoin 4-Aminosalicylic acid N-Acetyl-D-mannosamine L-Carnitine sn-glycero-3-Phosphocholine D-Proline Indole-3-carboxaldehyde L-Carnitine L-Carnitine L-Carnitine Creatine Cytidine Maltotriose L-Carnitine Cytidine-5'-monophosphate Trigonelline Uridine trans-Urocanic Acid L-Norleucine N-Acetylglycine L-Carnitine Thymine D-Leucine N-Acetylglutamic acid


/home/ernesto12/.local/lib/python3.10/site-packages/linear_operator-0.4.0-py3.10.egg/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  warnings.warn(
/home/ernesto12/.local/lib/python3.10/site-packages/linear_operator-0.4.0-py3.10.egg/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-05 to the diagonal
  warnings.warn(
/home/ernesto12/.local/lib/python3.10/site-packages/linear_operator-0.4.0-py3.10.egg/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  warnings.warn(
/home/ernesto12/.local/lib/python3.10/site-packages/linear_operator-0.4.0-py3.10.egg/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-05 to the diagonal
  warnings.warn(
/home/ernesto12/.local/lib/python3.10/site-packages/linear_operator-0.4.0-py3.10.egg/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added j